---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 4

In [12]:
import networkx as nx
import pandas as pd
import numpy as np
import pickle

---

## Part 1 - Random Graph Identification

For the first part of this assignment you will analyze randomly generated graphs and determine which algorithm created them.

In [2]:
P1_Graphs = pickle.load(open('A4_graphs','rb'))
P1_Graphs


<br>
`P1_Graphs` is a list containing 5 networkx graphs. Each of these graphs were generated by one of three possible algorithms:
* Preferential Attachment (`'PA'`)
* Small World with low probability of rewiring (`'SW_L'`)
* Small World with high probability of rewiring (`'SW_H'`)

Anaylze each of the 5 graphs and determine which of the three algorithms generated the graph.

*The `graph_identification` function should return a list of length 5 where each element in the list is either `'PA'`, `'SW_L'`, or `'SW_H'`.*

In [6]:
def histogram_list(G):
    degrees = G.degree()
    degree_values = sorted(set(degrees.values()))
    histogram = [list(degrees.values()).count(i)/float(nx.number_of_nodes(G)) for i in degree_values]
    return histogram

In [7]:
def graph_identification():
    graph_algorithms = []
    for graph in P1_Graphs:
        clustering = nx.average_clustering(graph)
        avg_short_path = nx.average_shortest_path_length(graph)
        degree_hist = histogram_list(graph)
        if len(degree_hist) > 10: 
            graph_algorithms.append('PA')
        elif clustering < 0.1:
            graph_algorithms.append('SW_H')
        else: 
            graph_algorithms.append('SW_L')
    return graph_algorithms
graph_identification()

['PA', 'SW_L', 'SW_L', 'PA', 'SW_H']

---

## Part 2 - Company Emails

For the second part of this assignment you will be workking with a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagementSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagementSalary` indicates whether that person is receiving a management position salary.

In [2]:
G = nx.read_gpickle('email_prediction.txt')

print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 1005
Number of edges: 16706
Average degree:  33.2458


In [8]:
G.nodes(data = True)[:10]

[(0, {'Department': 1, 'ManagementSalary': 0.0}),
 (1, {'Department': 1, 'ManagementSalary': nan}),
 (2, {'Department': 21, 'ManagementSalary': nan}),
 (3, {'Department': 21, 'ManagementSalary': 1.0}),
 (4, {'Department': 21, 'ManagementSalary': 1.0}),
 (5, {'Department': 25, 'ManagementSalary': nan}),
 (6, {'Department': 25, 'ManagementSalary': 1.0}),
 (7, {'Department': 14, 'ManagementSalary': 0.0}),
 (8, {'Department': 14, 'ManagementSalary': nan}),
 (9, {'Department': 14, 'ManagementSalary': 0.0})]

### Part 2A - Salary Prediction

Using network `G`, identify the people in the network with missing values for the node attribute `ManagementSalary` and predict whether or not these individuals are receiving a management position salary.

To accomplish this, you will need to create a matrix of node features using networkx, train a sklearn classifier on nodes that have `ManagementSalary` data, and predict a probability of the node receiving a management salary for nodes where `ManagementSalary` is missing.



Your predictions will need to be given as the probability that the corresponding employee is receiving a management position salary.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 252 with the data being the probability of receiving management salary, and the index being the node id.

    Example:
    
        1       1.0
        2       0.0
        5       0.8
        8       1.0
            ...
        996     0.7
        1000    0.5
        1001    0.0
        Length: 252, dtype: float64

In [19]:
def salary_predictions():
    from sklearn.svm import SVC
    from sklearn.neural_network import MLPClassifier
    from sklearn.preprocessing import MinMaxScaler
    df = pd.DataFrame(index = G.nodes())
    df['ManagementSalary'] = pd.Series(nx.get_node_attributes(G,'ManagementSalary'))
    df['Department'] = pd.Series(nx.get_node_attributes(G,'Department'))
    df['Degree'] = pd.Series(G.degree())
    df['Clustering'] = pd.Series(nx.clustering(G))
    df['Degree_centrality'] = pd.Series(nx.degree_centrality(G))
    df['Closeness'] = pd.Series(nx.closeness_centrality(G, normalized = True))
    df['Between'] = pd.Series(nx.betweenness_centrality(G,normalized = True))
    df['PR'] = pd.Series(nx.pagerank(G))
    
    df1 = df[df['ManagementSalary'].notnull()]
    X_train = df1.drop(['ManagementSalary'],axis = 1)      
    y_train = df1['ManagementSalary']   
    df2 = df[df['ManagementSalary'].isnull()]
    X_test = df2.drop(['ManagementSalary'],axis = 1)
    
    scaler = MinMaxScaler()
    X_train_transform = scaler.fit_transform(X_train)
    X_test_transform = scaler.transform(X_test)
    
    clf = MLPClassifier(hidden_layer_sizes = [10,10], solver = 'lbfgs',random_state = 0).fit(X_train_transform, y_train)
    y_pred = clf.predict_proba(X_test_transform)[:,1]
    return pd.Series(y_pred, index = X_test.index)
salary_predictions()

1       8.943934e-03
2       8.326531e-01
5       1.000000e+00
8       1.354277e-01
14      1.937863e-01
18      9.296487e-02
27      3.327030e-01
30      4.329390e-01
31      6.782873e-02
34      5.781506e-02
37      4.530626e-03
40      1.289006e-02
45      1.160351e-01
54      6.588442e-01
55      9.806466e-01
60      2.273892e-02
62      9.999960e-01
65      9.778071e-01
77      1.231481e-02
79      1.316356e-02
97      1.504960e-02
101     8.383971e-04
103     2.870448e-01
108     1.266395e-02
113     1.102188e-01
122     1.126633e-04
141     2.059336e-01
142     9.281364e-01
144     2.502971e-03
145     9.303182e-01
            ...     
913     1.343261e-04
914     2.153676e-03
915     2.317082e-03
918     1.298212e-01
923     4.505682e-06
926     6.107024e-03
931     3.887407e-04
934     2.262605e-07
939     1.679278e-07
944     2.984947e-03
945     2.241192e-05
947     1.519926e-01
950     3.647821e-02
951     1.108384e-05
953     1.456675e-05
959     6.301224e-03
962     5.185

### Part 2B - New Connections Prediction

For the last part of this assignment, you will predict future connections between employees of the network. The future connections information has been loaded into the variable `future_connections`. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

In [4]:
future_connections = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})
future_connections.head(10)

,Future Connection
"(6, 840)",0.0
"(4, 197)",0.0
"(620, 979)",0.0
"(519, 872)",0.0
"(382, 423)",0.0
"(97, 226)",1.0
"(349, 905)",0.0
"(429, 860)",0.0
"(309, 989)",0.0
"(468, 880)",0.0


Using network `G` and `future_connections`, identify the edges in `future_connections` with missing values and predict whether or not these edges will have a future connection.

To accomplish this, you will need to create a matrix of features for the edges found in `future_connections` using networkx, train a sklearn classifier on those edges in `future_connections` that have `Future Connection` data, and predict a probability of the edge being a future connection for those edges in `future_connections` where `Future Connection` is missing.



Your predictions will need to be given as the probability of the corresponding edge being a future connection.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 122112 with the data being the probability of the edge being a future connection, and the index being the edge as represented by a tuple of nodes.

    Example:
    
        (107, 348)    0.35
        (542, 751)    0.40
        (20, 426)     0.55
        (50, 989)     0.35
                  ...
        (939, 940)    0.15
        (555, 905)    0.35
        (75, 101)     0.65
        Length: 122112, dtype: float64

In [11]:
def new_connections_predictions():
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.preprocessing import MinMaxScaler
    
    future_connections['Preferential Attachment'] = [i[2] for i in nx.preferential_attachment(G, future_connections.index)]
    future_connections['Common Neighbours'] = [len(list(nx.common_neighbors(G, x[0], x[1]))) for x in future_connections.index]
    future_connections['Jaccard Coefficient'] = [i[2] for i in nx.jaccard_coefficient(G, future_connections.index)]
    future_connections['Resource Allocation'] = [i[2] for i in nx.resource_allocation_index(G, future_connections.index)]
    future_connections['Adamic Adar'] = [i[2] for i in nx.adamic_adar_index(G, future_connections.index)]
    for node in G.nodes():
        G.node[node]['community'] = G.node[node]['Department']
    future_connections['cn_soundarajan_hopcroft'] = [i[2] for i in nx.cn_soundarajan_hopcroft(G, future_connections.index)]   
    future_connections['ra_soundarajan_hopcroft'] = [i[2] for i in nx.ra_index_soundarajan_hopcroft(G, future_connections.index)]
    
    df1 = future_connections[future_connections['Future Connection'].notnull()]
    X_train = df1.drop(['Future Connection'],axis = 1)      
    y_train = df1['Future Connection']   
    df2 = future_connections[future_connections['Future Connection'].isnull()]
    X_test = df2.drop(['Future Connection'],axis = 1)
    
    scaler = MinMaxScaler()
    X_train_transform = scaler.fit_transform(X_train)
    X_test_transform = scaler.transform(X_test)
    clr = RandomForestClassifier(n_estimators = 50, max_depth = 10,random_state = 0).fit(X_train_transform, y_train)
    y_pred = clr.predict_proba(X_test_transform)[:,1]
    return pd.Series(y_pred, index = X_test.index)
new_connections_predictions()

(107, 348)    0.028398
(542, 751)    0.012462
(20, 426)     0.550567
(50, 989)     0.012538
(942, 986)    0.012663
(324, 857)    0.012538
(13, 710)     0.115066
(19, 271)     0.141415
(319, 878)    0.012644
(659, 707)    0.012499
(49, 843)     0.012634
(208, 893)    0.012499
(377, 469)    0.013810
(405, 999)    0.014537
(129, 740)    0.012593
(292, 618)    0.019572
(239, 689)    0.012319
(359, 373)    0.014896
(53, 523)     0.475549
(276, 984)    0.012276
(202, 997)    0.012672
(604, 619)    0.047547
(270, 911)    0.012319
(261, 481)    0.060666
(200, 450)    0.991549
(213, 634)    0.012379
(644, 735)    0.032205
(346, 553)    0.012187
(521, 738)    0.012007
(422, 953)    0.014648
                ...   
(672, 848)    0.012319
(28, 127)     0.988726
(202, 661)    0.012187
(54, 195)     1.000000
(295, 864)    0.012896
(814, 936)    0.012379
(839, 874)    0.012663
(139, 843)    0.012538
(461, 544)    0.013793
(68, 487)     0.013090
(622, 932)    0.012499
(504, 936)    0.017507
(479, 528) 